# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [67]:
# import libraries
import pandas as pd
from sklearn.pipeline import Pipeline,FeatureUnion
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.model_selection import GridSearchCV
import pickle
import warnings
warnings.filterwarnings('ignore')
import re

In [2]:
# load data from database
engine = create_engine('sqlite:///Disaster_Response.db')
df = pd.read_sql_table("Disaster_data",engine)
X = df['message']
y = df.iloc[:,4:]

In [3]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26210,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26211,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26212,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26213,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    url_format = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

    all_urls = re.findall(url_format,text)

    for url in all_urls:
        text = text.replace(url,"url_placeholder")

    tokenize_text = word_tokenize(text.lower())

    lemmatizer = WordNetLemmatizer()

    clean_tokens = []

    for tok in tokenize_text:
        token = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(token)

    return clean_tokens

In [5]:
tokenize("I think it is very pretty")

['i', 'think', 'it', 'is', 'very', 'pretty']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
class StartVerbExtractor(BaseEstimator,TransformerMixin):
    '''

    The StartVerbExtractor class consists of function starting_verb function 
    tokenizes the input text into sentences using 
    the nltk library, and then tags the first word in each phrase using part-of-speech 
    tagging. 

    The method returns True if the first word is a verb (as indicated by a 
    part-of-speech tag of 'VB' or 'VBP') or if it is the term 'RT' 
    (which may suggest a retweet). 

    If not, it returns False.

    '''

    def starting_verb(self, text):
        sentence_list = sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [8]:
def pipeline_classifier(clf, start_verb=None):

    if start_verb is None:

        pipeline = Pipeline([
            ('feature', FeatureUnion([
                ('txt_pipe', Pipeline([
                    ('vect', CountVectorizer(tokenizer=tokenize)),
                    ('tfidf', TfidfTransformer())
                ]))
            ])),
            ('clf', MultiOutputClassifier(clf))
        ])
        return pipeline
    
    else:
        pipeline = Pipeline([
        ('features', FeatureUnion([
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),
            ('starting_verb', StartVerbExtractor())
        ])),
        ('clf', MultiOutputClassifier(clf))
    ])
    return pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [18]:
ada = pipeline_classifier(AdaBoostClassifier(),start_verb=StartVerbExtractor())
ada.fit(X_train,y_train)
y_pred = ada.predict(X_test)

26.823314006713456


In [22]:
print(classification_report(y_test.values,y_pred,target_names=df.columns[4:]))

                        precision    recall  f1-score   support

               related       0.83      0.93      0.88      5023
               request       0.78      0.57      0.66      1111
                 offer       0.00      0.00      0.00        26
           aid_related       0.77      0.59      0.67      2717
          medical_help       0.64      0.29      0.40       516
      medical_products       0.60      0.30      0.40       319
     search_and_rescue       0.60      0.18      0.28       170
              security       0.19      0.03      0.05       129
              military       0.56      0.29      0.38       208
                 water       0.75      0.62      0.68       433
                  food       0.80      0.70      0.75       735
               shelter       0.76      0.55      0.64       575
              clothing       0.67      0.44      0.53        97
                 money       0.57      0.28      0.37       152
        missing_people       0.60      

In [25]:
ada.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'features', 'clf', 'features__n_jobs', 'features__transformer_list', 'features__transformer_weights', 'features__verbose', 'features__text_pipeline', 'features__starting_verb', 'features__text_pipeline__memory', 'features__text_pipeline__steps', 'features__text_pipeline__verbose', 'features__text_pipeline__vect', 'features__text_pipeline__tfidf', 'features__text_pipeline__vect__analyzer', 'features__text_pipeline__vect__binary', 'features__text_pipeline__vect__decode_error', 'features__text_pipeline__vect__dtype', 'features__text_pipeline__vect__encoding', 'features__text_pipeline__vect__input', 'features__text_pipeline__vect__lowercase', 'features__text_pipeline__vect__max_df', 'features__text_pipeline__vect__max_features', 'features__text_pipeline__vect__min_df', 'features__text_pipeline__vect__ngram_range', 'features__text_pipeline__vect__preprocessor', 'features__text_pipeline__vect__stop_words', 'features__text_pipeline__vect__strip_accents

### 6. Improve your model
Use grid search to find better parameters. 

### Applying Grid Search to the model 

In [28]:
model = pipeline_classifier(
    AdaBoostClassifier(), start_verb=StartVerbExtractor())

parameters = {
    'clf__estimator__learning_rate': [0.5, 1.0, 1.5],
    'clf__estimator__n_estimators': [5, 10, 15, 20]
}

gscv = GridSearchCV(model,param_grid=parameters,n_jobs=-1,cv=5,verbose=2)

gscv.fit(X_train,y_train)


Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 12.7min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('features',
                                        FeatureUnion(n_jobs=None,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(memory=None,
                                                                                 steps=[('vect',
                                                                                         CountVectorizer(analyzer='word',
                                                                                                         binary=False,
                                                                                                         decode_error='strict',
                                                                                                         dtype=<class 'numpy.int64'>,
                  

In [33]:
gscv.best_params_

{'clf__estimator__learning_rate': 1.0, 'clf__estimator__n_estimators': 20}

In [54]:
pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('txt_pipe', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(learning_rate=1.0,n_estimators=20)))
])

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('feature',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('txt_pipe',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  lowercase

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [29]:
category_names = list(df.columns[4:])

def evaluate_model(model, X_test, Y_test, category_names):

    y_pred = model.predict(X_test)

    print(classification_report(Y_test.values,y_pred,target_names=category_names))

In [60]:
evaluate_model(pipeline, X_test, y_test, category_names)

                        precision    recall  f1-score   support

               related       0.81      0.95      0.88      5023
               request       0.78      0.52      0.62      1111
                 offer       0.00      0.00      0.00        26
           aid_related       0.75      0.54      0.63      2717
          medical_help       0.63      0.27      0.37       516
      medical_products       0.64      0.28      0.39       319
     search_and_rescue       0.66      0.19      0.30       170
              security       0.17      0.02      0.03       129
              military       0.59      0.20      0.30       208
                 water       0.77      0.61      0.68       433
                  food       0.79      0.65      0.71       735
               shelter       0.77      0.47      0.58       575
              clothing       0.70      0.46      0.56        97
                 money       0.54      0.24      0.33       152
        missing_people       0.67      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [62]:
pipeline = Pipeline([
    ('feature', FeatureUnion([
        ('txt_pipe', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ]))
    ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=20)))
])

pipeline.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('feature',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('txt_pipe',
                                                 Pipeline(memory=None,
                                                          steps=[('vect',
                                                                  CountVectorizer(analyzer='word',
                                                                                  binary=False,
                                                                                  decode_error='strict',
                                                                                  dtype=<class 'numpy.int64'>,
                                                                                  encoding='utf-8',
                                                                                  input='content',
                                                                                  lowercase

In [65]:
y_preds = pipeline.predict(X_test)

In [66]:
print(classification_report(y_test.values,y_preds,target_names=category_names))

                        precision    recall  f1-score   support

               related       0.83      0.96      0.89      5052
               request       0.85      0.40      0.55      1160
                 offer       0.00      0.00      0.00        27
           aid_related       0.77      0.55      0.64      2769
          medical_help       0.61      0.05      0.09       513
      medical_products       0.80      0.11      0.20       352
     search_and_rescue       0.67      0.01      0.02       182
              security       0.00      0.00      0.00       122
              military       0.54      0.03      0.06       221
                 water       0.87      0.28      0.43       423
                  food       0.83      0.45      0.58       754
               shelter       0.82      0.26      0.40       570
              clothing       0.67      0.09      0.16        87
                 money       0.67      0.03      0.05       148
        missing_people       0.50      

### 9. Export your model as a pickle file

In [68]:
with open('classifier.pkl','wb') as f:
    pickle.dump(gscv,f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.